In [1]:
from pytorch_pae import AE

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import torch

print(torch.__version__)

1.8.0


In [10]:
from torchsummary import summary

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
## todo: add bias 

In [6]:
# number of layers in networks
n_layers     = 3

## convolutional net specific parameters
# number of channels in each layer for convolutional neural net
out_channels = [16,16,16]
# kernel sizes in each layer for conv net
kernel_sizes = [4,3,2]
# scaling factor in max pooling layer 
scale_facs   = [1,1,1] 
# padding values in each conv layer
paddings     = [0,0,0]
# stride values in each conv layer
strides      = [2,1,1]
# whether tp apply a layer normalization after conv layer
layer_norm   = [True,True,True]
# whether to train elemntwise affine parameters for normalization layer 
affine       = False

## fully connected net specific parameters
# output size of each fully connected layer
out_sizes    = [256,128,64]

## parameters that apply to both, fully connected and convolutional nets 
# dropout rate after each layer
dropout_rate = [0.,0.,0.]
# whether to Lipschitz regularize by bounding the spectral norm 
spec_norm    = True
# activation function after each layer
activations  = ['ReLU', 'ReLU','ReLU']
# whether to add a bias in each layer or not
bias         = [True, True, True]

## general parameters
# data dimensionality
dim          = '2D'
# latent space dimensionality
latent_dim   = 8
# number of channels in data
input_c      = 1 
# data dimensioality along one axis (only square data supported in 2D)
input_dim    = 28
# type of encoder and decoder network (either 'fc' or 'conv')
encoder_type = 'conv'
decoder_type = 'conv'

# if True, the output is fed through a sigmoid layer to bring data values into range [0,1]
final_sigmoid = True

nepochs       = 20
batchsize     = 64
initial_lr    = 1e-2

optimizer     = 'Adam'
criterion     = 'MSELoss'

scheduler     = 'ExponentialLR'
scheduler_params = {'gamma':0.95}

dataset       = 'MNIST'

In [7]:
general_params      = {'input_c': input_c, 'input_dim': input_dim, 'latent_dim': latent_dim, 'encoder_type': encoder_type, 'decoder_type': decoder_type, 'dim': dim}
conv_network_params = {'n_layers': n_layers, 'out_channels': out_channels, 'kernel_sizes': kernel_sizes, 'scale_facs': scale_facs, 'paddings': paddings,\
                       'strides': strides,'activations': activations, 'spec_norm': spec_norm, 'dropout_rate':dropout_rate, 'layer_norm': layer_norm,\
                       'affine': affine,'final_sigmoid': final_sigmoid}
fc_network_params   = {'n_layers': n_layers, 'out_sizes': out_sizes,'activations': activations, 'spec_norm': spec_norm, 'dropout_rate':dropout_rate, \
                       'layer_norm': layer_norm, 'affine': affine, 'final_sigmoid': final_sigmoid}

training_params     = {'batchsize': batchsize, 'initial_lr': initial_lr, 'optimizer': optimizer, 'criterion': criterion, \
                       'scheduler': scheduler, 'scheduler_params':scheduler_params}
data_params         = {'dataset':dataset, 'loc': '/global/cscratch1/sd/vboehm/Datasets'}

In [8]:
AE = AE.Autoencoder(general_params,data_params,conv_network_params, conv_network_params, training_params, device)

In [11]:
summary(AE, (1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 13, 13]             272
         LayerNorm-2           [-1, 16, 13, 13]               0
              ReLU-3           [-1, 16, 13, 13]               0
 AdaptiveMaxPool2d-4           [-1, 16, 13, 13]               0
            Conv2d-5           [-1, 16, 11, 11]           2,320
         LayerNorm-6           [-1, 16, 11, 11]               0
              ReLU-7           [-1, 16, 11, 11]               0
 AdaptiveMaxPool2d-8           [-1, 16, 11, 11]               0
            Conv2d-9           [-1, 16, 10, 10]           1,040
        LayerNorm-10           [-1, 16, 10, 10]               0
             ReLU-11           [-1, 16, 10, 10]               0
AdaptiveMaxPool2d-12           [-1, 16, 10, 10]               0
          Flatten-13                 [-1, 1600]               0
           Linear-14                   

In [9]:
train_loss, valid_loss = AE.train(10)

epoch: 0, training loss: 3.0705e-02, validation loss: 2.2565e-02, learning rate: 9.5000e-03
epoch: 1, training loss: 1.9731e-02, validation loss: 1.7925e-02, learning rate: 9.0250e-03
epoch: 2, training loss: 1.8429e-02, validation loss: 1.8713e-02, learning rate: 8.5737e-03
epoch: 3, training loss: 1.7735e-02, validation loss: 1.7695e-02, learning rate: 8.1451e-03
epoch: 4, training loss: 1.7255e-02, validation loss: 1.6603e-02, learning rate: 7.7378e-03
epoch: 5, training loss: 1.6942e-02, validation loss: 1.7432e-02, learning rate: 7.3509e-03
epoch: 6, training loss: 1.6647e-02, validation loss: 1.6894e-02, learning rate: 6.9834e-03
epoch: 7, training loss: 1.6478e-02, validation loss: 1.5405e-02, learning rate: 6.6342e-03
epoch: 8, training loss: 1.6249e-02, validation loss: 1.6336e-02, learning rate: 6.3025e-03
epoch: 9, training loss: 1.6109e-02, validation loss: 1.4930e-02, learning rate: 5.9874e-03


([0.03070538523015561,
  0.019730683540370764,
  0.018428563739679032,
  0.017734579952670262,
  0.017254996642255667,
  0.016942056769598573,
  0.01664743544245606,
  0.016477843620312443,
  0.016249115251823702,
  0.016109050875738374],
 [tensor(0.0226, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.0179, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.0187, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.0177, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.0166, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.0174, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.0169, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.0154, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.0163, device='cuda:0', grad_fn=<MseLossBackward>),
  tensor(0.0149, device='cuda:0', grad_fn=<MseLossBackward>)])

In [10]:
import pytorch_pae.utils as utils

In [11]:
utils.output_shape

<function pytorch_pae.utils.output_shape(in_dim, stride, padding, kernel, dilation=1)>